In [1]:
## This file implements neural networks and logistic regression on p0017presabsSTCC_quant.
## For fully-connected neural networks, the accuracy is 58.44%.
## To improve accuracy, we construct lasso regression to select important features, and thus form a new dataset.
## With the new dataset, the accuracy is 59.74% after improvement.
## For logistic regression, the accuracy is 61.04%.
## We further construct random forest models, which are relatively less likely to bring overfitting 
## compared to decision tree.
## For random forest, the accuracy is 63.64%.
## For random forest with cross-validation, the mean accuracy is 77.48%.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p0017presabsSTCC_quant.csv')
df.shape

(255, 7006)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      0.871375
1      0.364375
2      0.803000
3      0.952625
4      1.012875
5      0.936750
6      0.745375
7      0.348375
8      0.571750
9      0.637125
10     0.670500
11     0.731250
12     0.865125
13     0.569375
14     0.327000
15     0.175750
16     0.964000
17     1.104167
18     0.223667
19     0.340167
20     0.438500
21     0.271833
22     0.248833
23     0.739333
24     0.232833
25     0.601500
26     0.251000
27     0.202833
28     0.599833
29     0.260667
         ...   
225    0.459500
226    0.825333
227    0.763167
228    0.334500
229    0.998833
230    0.491125
231    0.405750
232    0.875375
233    0.761500
234    0.255000
235    0.455125
236    0.234125
237    0.340125
238    0.952750
239    0.238500
240    0.294500
241    0.172125
242    0.221500
243    0.989875
244    0.763000
245    0.229875
246    0.215125
247    0.805375
248    0.184875
249    0.663250
250    0.803125
251    0.443875
252    0.207500
253    0.933875
254    0.873375
Name: pheno, Length: 255

In [6]:
df.head()

,id,TTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAG,TTTTTTTGTAAATCTTTTCCATTTATTTCTTCGCTTTTCTCGCTACAATTTACAAATGGACTTGTTATCAACGTTAGGGCAATAAATGAAAGTAGTATAA,TTTTTTTGATAATTT,TTTTTTTCTTATTATACGAAATTTTCATATCATAGTAAAGTTTTTTACGAAAAAAACGTATTTAATGTTGACAATACGAAAATTTCGTATTATATTAGGT,TTTTTTTCGTAAAAAACTTTACTATGATATGAAAATTTCGTATAATAAGAAAAAAAGGAGGTAAGTAATATGAACAAAGAAAGAAATATTATTATAGCCA,TTTTTTTATTTTGCAATTTTTTATTTTCATTATAAACTTCCTTTCAAACACTGCTGAAATAGACGTCTTTTTCAAATAAGCATGATTAATACTTCAATTC,TTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAGT,TTTTTTTATGTTATTATAATTAAAGTTTTCCATTGTTTTCCTCCTATAATAGCTTATCTGCAATCATCACAGCTAATAAATCGTTTTGTCTTATTGCTTC,TTTTTTTATGTTATAATCTTTCTAGACGTATTCAAAGGACGTCTTTTTAGATTGTATGTTATAGCTAGCCTTCCGGTTAATTTTTTGTTATGATGTGTTA,...,group_426,group_4770,group_8913,group_9224,group_9493,group_9494,group_9905,pheno,ST,CC
0,107,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.871375,5,5
1,109,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.364375,8,8
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.803000,5,5
3,120335,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0.952625,5,5
4,120337,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1.012875,5,5


In [7]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [8]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [9]:
df['pheno']
df['pheno'].value_counts()

1    135
0    120
Name: pheno, dtype: int64

In [10]:
df.shape

(255, 7006)

In [11]:
df_clean = df.drop(columns=['id'])

In [12]:
df_clean.shape

(255, 7005)

In [13]:
df_clean.head()

,TTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAG,TTTTTTTGTAAATCTTTTCCATTTATTTCTTCGCTTTTCTCGCTACAATTTACAAATGGACTTGTTATCAACGTTAGGGCAATAAATGAAAGTAGTATAA,TTTTTTTGATAATTT,TTTTTTTCTTATTATACGAAATTTTCATATCATAGTAAAGTTTTTTACGAAAAAAACGTATTTAATGTTGACAATACGAAAATTTCGTATTATATTAGGT,TTTTTTTCGTAAAAAACTTTACTATGATATGAAAATTTCGTATAATAAGAAAAAAAGGAGGTAAGTAATATGAACAAAGAAAGAAATATTATTATAGCCA,TTTTTTTATTTTGCAATTTTTTATTTTCATTATAAACTTCCTTTCAAACACTGCTGAAATAGACGTCTTTTTCAAATAAGCATGATTAATACTTCAATTC,TTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAGT,TTTTTTTATGTTATTATAATTAAAGTTTTCCATTGTTTTCCTCCTATAATAGCTTATCTGCAATCATCACAGCTAATAAATCGTTTTGTCTTATTGCTTC,TTTTTTTATGTTATAATCTTTCTAGACGTATTCAAAGGACGTCTTTTTAGATTGTATGTTATAGCTAGCCTTCCGGTTAATTTTTTGTTATGATGTGTTA,TTTTTTTATAACATTCAAAGTCTCACCATTGTCATTTGAATGATCATCAATAATAATTAATTCGTAATCAGTACTCTTCATTGTTTGATTTAATACAGAA,...,group_426,group_4770,group_8913,group_9224,group_9493,group_9494,group_9905,pheno,ST,CC
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,5,5
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8,8
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,5,5
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,5,5
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,5,5


In [14]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 7004) (255,)


In [15]:
# split into train, validation, and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y)

In [16]:
############# Fully-Connected Neural Network ################

In [17]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

Using TensorFlow backend.


In [18]:
#### neural network
model1 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [19]:
model1.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [20]:
model1.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 178 samples, validate on 77 samples
Epoch 1/100
178/178 [==============================] - 0s 2ms/step - loss: 13.9673 - accuracy: 0.5281 - val_loss: 31.8429 - val_accuracy: 0.5455
Epoch 2/100
178/178 [==============================] - 0s 363us/step - loss: 5.9862 - accuracy: 0.5787 - val_loss: 0.6472 - val_accuracy: 0.5325
Epoch 3/100
178/178 [==============================] - 0s 453us/step - loss: 0.6208 - accuracy: 0.7135 - val_loss: 0.6436 - val_accuracy: 0.5195
Epoch 4/100
178/178 [==============================] - 0s 487us/step - loss: 0.6240 - accuracy: 0.7247 - val_loss: 0.6464 - val_accuracy: 0.5195
Epoch 5/100
178/178 [==============================] - 0s 362us/step - loss: 0.5895 - accuracy: 0.6910 - val_loss: 0.6530 - val_accuracy: 0.5065
Epoch 6/100
178/178 [==============================] - 0s 331us/step - loss: 0.5790 - accuracy: 0.7079 - val_loss: 0.6406 - val_accuracy: 0.5325
Epoch 7/100
178/178 [==============================] - 0s 366us/step - loss: 0.5734 -

178/178 [==============================] - 0s 275us/step - loss: 0.4432 - accuracy: 0.8371 - val_loss: 0.6653 - val_accuracy: 0.6364
Epoch 57/100
178/178 [==============================] - 0s 247us/step - loss: 0.4456 - accuracy: 0.8146 - val_loss: 0.6563 - val_accuracy: 0.5974
Epoch 58/100
178/178 [==============================] - 0s 240us/step - loss: 0.4475 - accuracy: 0.8090 - val_loss: 0.6579 - val_accuracy: 0.5974
Epoch 59/100
178/178 [==============================] - 0s 215us/step - loss: 0.4424 - accuracy: 0.8371 - val_loss: 0.6712 - val_accuracy: 0.5844
Epoch 60/100
178/178 [==============================] - 0s 225us/step - loss: 0.4381 - accuracy: 0.8371 - val_loss: 0.6649 - val_accuracy: 0.6234
Epoch 61/100
178/178 [==============================] - 0s 220us/step - loss: 0.4471 - accuracy: 0.8034 - val_loss: 0.6615 - val_accuracy: 0.5844
Epoch 62/100
178/178 [==============================] - 0s 219us/step - loss: 0.4365 - accuracy: 0.8427 - val_loss: 0.6742 - val_accuracy

In [21]:
acc_test = model1.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test*100))

77/77 [==============================] - 0s 155us/step
test accuracy: 57.14%


In [55]:
#### improve neural network
model2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [56]:
model2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [57]:
model2.fit(X_train, y_train,
          batch_size=64, epochs=1000,
          validation_data=(X_test, y_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 739us/step - loss: 3.6093 - accuracy: 0.5618 - val_loss: 3.6606 - val_accuracy: 0.5325
Epoch 2/1000
178/178 [==============================] - 0s 186us/step - loss: 4.9202 - accuracy: 0.5787 - val_loss: 8.8623 - val_accuracy: 0.5455
Epoch 3/1000
178/178 [==============================] - 0s 176us/step - loss: 5.3127 - accuracy: 0.5449 - val_loss: 1.2066 - val_accuracy: 0.4416
Epoch 4/1000
178/178 [==============================] - 0s 192us/step - loss: 2.4927 - accuracy: 0.5674 - val_loss: 3.8061 - val_accuracy: 0.5325
Epoch 5/1000
178/178 [==============================] - 0s 164us/step - loss: 3.7698 - accuracy: 0.5449 - val_loss: 1.8662 - val_accuracy: 0.5325
Epoch 6/1000
178/178 [==============================] - 0s 175us/step - loss: 2.1973 - accuracy: 0.6067 - val_loss: 1.5512 - val_accuracy: 0.5195
Epoch 7/1000
178/178 [==============================] - 0s 190us/step - loss: 1

178/178 [==============================] - 0s 189us/step - loss: 0.5649 - accuracy: 0.7247 - val_loss: 0.6385 - val_accuracy: 0.5714
Epoch 57/1000
178/178 [==============================] - 0s 170us/step - loss: 0.5634 - accuracy: 0.7135 - val_loss: 0.6395 - val_accuracy: 0.5714
Epoch 58/1000
178/178 [==============================] - 0s 162us/step - loss: 0.5620 - accuracy: 0.7135 - val_loss: 0.6397 - val_accuracy: 0.5584
Epoch 59/1000
178/178 [==============================] - 0s 173us/step - loss: 0.5603 - accuracy: 0.7135 - val_loss: 0.6389 - val_accuracy: 0.5714
Epoch 60/1000
178/178 [==============================] - 0s 173us/step - loss: 0.5587 - accuracy: 0.7079 - val_loss: 0.6397 - val_accuracy: 0.5455
Epoch 61/1000
178/178 [==============================] - 0s 193us/step - loss: 0.5575 - accuracy: 0.7135 - val_loss: 0.6398 - val_accuracy: 0.5584
Epoch 62/1000
178/178 [==============================] - 0s 171us/step - loss: 0.5557 - accuracy: 0.7303 - val_loss: 0.6394 - val_ac

Epoch 112/1000
178/178 [==============================] - 0s 146us/step - loss: 0.4902 - accuracy: 0.7753 - val_loss: 0.6383 - val_accuracy: 0.5714
Epoch 113/1000
178/178 [==============================] - 0s 159us/step - loss: 0.4885 - accuracy: 0.7978 - val_loss: 0.6373 - val_accuracy: 0.5584
Epoch 114/1000
178/178 [==============================] - 0s 160us/step - loss: 0.4869 - accuracy: 0.8034 - val_loss: 0.6375 - val_accuracy: 0.5584
Epoch 115/1000
178/178 [==============================] - 0s 163us/step - loss: 0.4876 - accuracy: 0.7809 - val_loss: 0.6368 - val_accuracy: 0.5584
Epoch 116/1000
178/178 [==============================] - 0s 149us/step - loss: 0.4852 - accuracy: 0.8034 - val_loss: 0.6386 - val_accuracy: 0.5974
Epoch 117/1000
178/178 [==============================] - 0s 159us/step - loss: 0.4856 - accuracy: 0.8034 - val_loss: 0.6371 - val_accuracy: 0.5714
Epoch 118/1000
178/178 [==============================] - 0s 160us/step - loss: 0.4822 - accuracy: 0.8090 - val_

178/178 [==============================] - 0s 152us/step - loss: 0.4327 - accuracy: 0.8371 - val_loss: 0.6469 - val_accuracy: 0.5974
Epoch 168/1000
178/178 [==============================] - 0s 167us/step - loss: 0.4320 - accuracy: 0.8258 - val_loss: 0.6458 - val_accuracy: 0.5974
Epoch 169/1000
178/178 [==============================] - 0s 164us/step - loss: 0.4301 - accuracy: 0.8315 - val_loss: 0.6451 - val_accuracy: 0.5974
Epoch 170/1000
178/178 [==============================] - 0s 171us/step - loss: 0.4312 - accuracy: 0.8371 - val_loss: 0.6432 - val_accuracy: 0.6104
Epoch 171/1000
178/178 [==============================] - 0s 164us/step - loss: 0.4296 - accuracy: 0.8315 - val_loss: 0.6526 - val_accuracy: 0.6234
Epoch 172/1000
178/178 [==============================] - 0s 148us/step - loss: 0.4293 - accuracy: 0.8371 - val_loss: 0.6485 - val_accuracy: 0.5974
Epoch 173/1000
178/178 [==============================] - 0s 175us/step - loss: 0.4302 - accuracy: 0.8258 - val_loss: 0.6540 - 

178/178 [==============================] - 0s 168us/step - loss: 0.3901 - accuracy: 0.8427 - val_loss: 0.6837 - val_accuracy: 0.5974
Epoch 223/1000
178/178 [==============================] - 0s 182us/step - loss: 0.3898 - accuracy: 0.8371 - val_loss: 0.6824 - val_accuracy: 0.5974
Epoch 224/1000
178/178 [==============================] - 0s 177us/step - loss: 0.3902 - accuracy: 0.8258 - val_loss: 0.6668 - val_accuracy: 0.6234
Epoch 225/1000
178/178 [==============================] - 0s 179us/step - loss: 0.3845 - accuracy: 0.8315 - val_loss: 0.6820 - val_accuracy: 0.5974
Epoch 226/1000
178/178 [==============================] - 0s 137us/step - loss: 0.4671 - accuracy: 0.8371 - val_loss: 0.6892 - val_accuracy: 0.6104
Epoch 227/1000
178/178 [==============================] - 0s 156us/step - loss: 0.4085 - accuracy: 0.8258 - val_loss: 0.6795 - val_accuracy: 0.5974
Epoch 228/1000
178/178 [==============================] - 0s 166us/step - loss: 0.4015 - accuracy: 0.8258 - val_loss: 0.6829 - 

...

178/178 [==============================] - 0s 169us/step - loss: 0.2254 - accuracy: 0.8876 - val_loss: 1.0373 - val_accuracy: 0.5844
Epoch 773/1000
178/178 [==============================] - 0s 161us/step - loss: 0.2316 - accuracy: 0.8989 - val_loss: 1.0964 - val_accuracy: 0.5974
Epoch 774/1000
178/178 [==============================] - 0s 157us/step - loss: 0.2421 - accuracy: 0.8876 - val_loss: 1.0657 - val_accuracy: 0.5974
Epoch 775/1000
178/178 [==============================] - 0s 137us/step - loss: 0.2261 - accuracy: 0.8989 - val_loss: 1.0461 - val_accuracy: 0.5714
Epoch 776/1000
178/178 [==============================] - 0s 160us/step - loss: 0.2248 - accuracy: 0.8933 - val_loss: 1.0517 - val_accuracy: 0.5714
Epoch 777/1000
178/178 [==============================] - 0s 169us/step - loss: 0.2249 - accuracy: 0.8933 - val_loss: 1.0423 - val_accuracy: 0.5844
Epoch 778/1000
178/178 [==============================] - 0s 168us/step - loss: 0.2292 - accuracy: 0.8989 - val_loss: 1.0504 - 

178/178 [==============================] - 0s 172us/step - loss: 0.2180 - accuracy: 0.8989 - val_loss: 1.0661 - val_accuracy: 0.5974
Epoch 828/1000
178/178 [==============================] - 0s 170us/step - loss: 0.2188 - accuracy: 0.8989 - val_loss: 1.0570 - val_accuracy: 0.5844
Epoch 829/1000
178/178 [==============================] - 0s 175us/step - loss: 0.2163 - accuracy: 0.9045 - val_loss: 1.0546 - val_accuracy: 0.5844
Epoch 830/1000
178/178 [==============================] - 0s 179us/step - loss: 0.2188 - accuracy: 0.8989 - val_loss: 1.0455 - val_accuracy: 0.5844
Epoch 831/1000
178/178 [==============================] - 0s 179us/step - loss: 0.2224 - accuracy: 0.9045 - val_loss: 1.0402 - val_accuracy: 0.5844
Epoch 832/1000
178/178 [==============================] - 0s 182us/step - loss: 0.2212 - accuracy: 0.8989 - val_loss: 1.0469 - val_accuracy: 0.5844
Epoch 833/1000
178/178 [==============================] - 0s 174us/step - loss: 0.2215 - accuracy: 0.8876 - val_loss: 1.0517 - 

178/178 [==============================] - 0s 185us/step - loss: 0.2183 - accuracy: 0.8989 - val_loss: 1.0998 - val_accuracy: 0.5714
Epoch 883/1000
178/178 [==============================] - 0s 201us/step - loss: 0.2293 - accuracy: 0.8876 - val_loss: 1.0863 - val_accuracy: 0.5844
Epoch 884/1000
178/178 [==============================] - 0s 138us/step - loss: 0.2263 - accuracy: 0.9045 - val_loss: 1.0765 - val_accuracy: 0.5974
Epoch 885/1000
178/178 [==============================] - 0s 170us/step - loss: 0.2228 - accuracy: 0.8989 - val_loss: 1.0861 - val_accuracy: 0.5844
Epoch 886/1000
178/178 [==============================] - 0s 187us/step - loss: 0.2259 - accuracy: 0.8989 - val_loss: 1.1167 - val_accuracy: 0.5714
Epoch 887/1000
178/178 [==============================] - 0s 178us/step - loss: 0.2200 - accuracy: 0.8989 - val_loss: 1.0843 - val_accuracy: 0.5974
Epoch 888/1000
178/178 [==============================] - 0s 166us/step - loss: 0.2268 - accuracy: 0.9045 - val_loss: 1.0854 - 

178/178 [==============================] - 0s 183us/step - loss: 0.2197 - accuracy: 0.8989 - val_loss: 1.1206 - val_accuracy: 0.5714
Epoch 938/1000
178/178 [==============================] - 0s 182us/step - loss: 0.2159 - accuracy: 0.9045 - val_loss: 1.1181 - val_accuracy: 0.5844
Epoch 939/1000
178/178 [==============================] - 0s 187us/step - loss: 0.2111 - accuracy: 0.9045 - val_loss: 1.0972 - val_accuracy: 0.5844
Epoch 940/1000
178/178 [==============================] - 0s 164us/step - loss: 0.2152 - accuracy: 0.9045 - val_loss: 1.0954 - val_accuracy: 0.5844
Epoch 941/1000
178/178 [==============================] - 0s 142us/step - loss: 0.2257 - accuracy: 0.8933 - val_loss: 1.1291 - val_accuracy: 0.5844
Epoch 942/1000
178/178 [==============================] - 0s 167us/step - loss: 0.2121 - accuracy: 0.9045 - val_loss: 1.1179 - val_accuracy: 0.5974
Epoch 943/1000
178/178 [==============================] - 0s 188us/step - loss: 0.2151 - accuracy: 0.8989 - val_loss: 1.1086 - 

178/178 [==============================] - 0s 172us/step - loss: 0.2107 - accuracy: 0.8989 - val_loss: 1.1847 - val_accuracy: 0.5974
Epoch 993/1000
178/178 [==============================] - 0s 187us/step - loss: 0.2114 - accuracy: 0.8989 - val_loss: 1.1572 - val_accuracy: 0.5844
Epoch 994/1000
178/178 [==============================] - 0s 159us/step - loss: 0.2148 - accuracy: 0.8989 - val_loss: 1.2442 - val_accuracy: 0.5974
Epoch 995/1000
178/178 [==============================] - 0s 164us/step - loss: 0.2178 - accuracy: 0.8989 - val_loss: 1.1269 - val_accuracy: 0.5974
Epoch 996/1000
178/178 [==============================] - 0s 181us/step - loss: 0.2122 - accuracy: 0.9045 - val_loss: 1.1183 - val_accuracy: 0.5844
Epoch 997/1000
178/178 [==============================] - 0s 188us/step - loss: 0.2088 - accuracy: 0.8989 - val_loss: 1.1342 - val_accuracy: 0.5844
Epoch 998/1000
178/178 [==============================] - 0s 146us/step - loss: 0.2119 - accuracy: 0.9045 - val_loss: 1.1247 - 

In [58]:
acc_test2 = model2.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test2*100))

77/77 [==============================] - 0s 206us/step
test accuracy: 58.44%


In [26]:
############ Feature selection using lasso ##########

In [27]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [28]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
selection.fit(X_train, y_train)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [29]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [30]:
X_train_features = np.vstack((names, X_train))
X_train_features = pd.DataFrame(X_train_features)

In [31]:
sel_features = X_train_features.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 7004
selected features: 112


In [32]:
cols = sel_features.values
cols.reshape((1, -1))

array([[ 354,  403,  445,  570,  672,  861,  874,  910,  991,  994,  998,
        1372, 1576, 1606, 1947, 2089, 2104, 2106, 2235, 2283, 2292, 2319,
        2326, 2481, 2511, 2520, 2546, 2587, 2630, 2666, 2804, 2830, 2997,
        3013, 3109, 3173, 3354, 3584, 3586, 3613, 3623, 3686, 3752, 3817,
        3825, 3874, 4031, 4047, 4096, 4227, 4278, 4344, 4345, 4350, 4449,
        4463, 4544, 4545, 4568, 4623, 4645, 4665, 4736, 4738, 4762, 4842,
        5047, 5068, 5100, 5133, 5143, 5189, 5382, 5492, 5543, 5639, 5658,
        5745, 5758, 5890, 5955, 5982, 6005, 6024, 6067, 6192, 6215, 6393,
        6421, 6424, 6533, 6547, 6581, 6596, 6616, 6745, 6755, 6914, 6926,
        6933, 6935, 6941, 6960, 6965, 6972, 6976, 6980, 6983, 6984, 6986,
        7002, 7003]])

In [33]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTGGAACAAAAC',
       'TTTGAATGCTAACGTAACTATTTCCATCTCTTCGTTTAACTAAGTTGAATTTGCCCATACCTGATTCAATTGTAGTGTCTCCGCCTGTAGCATAGTCACC',
       'TTTCTAGGTACG', 'TTTATCTATATCA',
       'TTTACAATTGCTTATAAGAAATCTGAAACATTTAGAAATTTTGTTAATGGTGCAATTGAAAGTGTTAAACAAACATTTAGTAATTTTATTCAATTTATTC',
       'TTGGAGTAATT',
       'TTGCTTATAAGAAATCTGAAACATTTAGAAATTTTGTTAATGGTGCAATTGAAAGTGTTAAACAAACATTTAGTAATTTTATTCAATTTATTCAACCTTT',
       'TTGCCAGGATA',
       'TTGAATGCTAACGTAACTATTTCCATCTCTTCGTTTAACTAAGTTGAATTTGCCCATACCTGATTCAATTGTAGTGTCTCCGCCTGTAGCATAGTCACCA',
       'TTGAATCAGGTATGGGCAAATTCAACTTAGTTAAACGAAGAGATGGAAATAGTTACGTTAGCATTCAAAGTTATGATTTATTGGCGGTAGGTTCTGATAA',
       'TTGAATAAATTGAATAAAATTACTAAATGTTTGTTTAACACTTTCAATTGCACCATTAACAAAATTTCTAAATGTTTCAGATTTCTTATAAGCAATTGTA',
       'TTATCCGAAATTT',
       'TTACAATTGCTTATAAGAAATCTGAAACATTTAGAAATTTTGTTAATGGTGCAATTGAAAGTGTTAAACAAACATTTAGTAATTTTATTCAATTTATTCA',
       'TTAATGGTGACTATGCTACAGGCGGAGACACTACAATTGAATCAGGTATGGGCAAATTCAACTTAGTTAAACGAAGAGATGGA

In [34]:
removed_features = X_train_features.columns[(selection.estimator_.coef_ == 0).ravel().tolist()]
removed_features

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            6992, 6993, 6994, 6995, 6996, 6997, 6998, 6999, 7000, 7001],
           dtype='int64', length=6892)

In [35]:
len(removed_features)

6892

In [36]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']
X_sel = df_sel.loc[:, df_sel.columns != 'pheno'].values
y_sel = df_sel['pheno'].values
print(X_sel.shape, y_sel.shape, df_sel.shape)

(255, 112) (255,) (255, 113)


In [37]:
# split into train, validation, and test data
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel)

In [38]:
#### neural network
model_sel1 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [39]:
model_sel1.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [40]:
model_sel1.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 810us/step - loss: 8.3660 - accuracy: 0.5506 - val_loss: 2.5218 - val_accuracy: 0.4545
Epoch 2/1000
178/178 [==============================] - 0s 82us/step - loss: 1.1156 - accuracy: 0.5169 - val_loss: 0.6902 - val_accuracy: 0.5714
Epoch 3/1000
178/178 [==============================] - 0s 84us/step - loss: 1.1946 - accuracy: 0.5899 - val_loss: 0.6740 - val_accuracy: 0.5844
Epoch 4/1000
178/178 [==============================] - 0s 146us/step - loss: 0.6541 - accuracy: 0.5562 - val_loss: 0.6718 - val_accuracy: 0.5974
Epoch 5/1000
178/178 [==============================] - 0s 181us/step - loss: 0.6553 - accuracy: 0.5618 - val_loss: 0.7001 - val_accuracy: 0.5584
Epoch 6/1000
178/178 [==============================] - 0s 146us/step - loss: 0.6682 - accuracy: 0.5899 - val_loss: 0.6680 - val_accuracy: 0.5714
Epoch 7/1000
178/178 [==============================] - 0s 195us/step - loss: 0.6

Epoch 57/1000
178/178 [==============================] - 0s 117us/step - loss: 0.5667 - accuracy: 0.7360 - val_loss: 0.6872 - val_accuracy: 0.5584
Epoch 58/1000
178/178 [==============================] - 0s 91us/step - loss: 0.5826 - accuracy: 0.7191 - val_loss: 0.6860 - val_accuracy: 0.5714
Epoch 59/1000
178/178 [==============================] - 0s 99us/step - loss: 0.5684 - accuracy: 0.7247 - val_loss: 0.6756 - val_accuracy: 0.5714
Epoch 60/1000
178/178 [==============================] - 0s 106us/step - loss: 0.5539 - accuracy: 0.7472 - val_loss: 0.6811 - val_accuracy: 0.5714
Epoch 61/1000
178/178 [==============================] - 0s 134us/step - loss: 0.5570 - accuracy: 0.7584 - val_loss: 0.6694 - val_accuracy: 0.5714
Epoch 62/1000
178/178 [==============================] - 0s 113us/step - loss: 0.5430 - accuracy: 0.7528 - val_loss: 0.6675 - val_accuracy: 0.5714
Epoch 63/1000
178/178 [==============================] - 0s 136us/step - loss: 0.5479 - accuracy: 0.7360 - val_loss: 0.6

Epoch 113/1000
178/178 [==============================] - 0s 114us/step - loss: 0.5158 - accuracy: 0.7640 - val_loss: 0.7489 - val_accuracy: 0.5844
Epoch 114/1000
178/178 [==============================] - 0s 110us/step - loss: 0.5033 - accuracy: 0.7640 - val_loss: 0.6997 - val_accuracy: 0.5974
Epoch 115/1000
178/178 [==============================] - 0s 125us/step - loss: 0.5192 - accuracy: 0.7753 - val_loss: 0.6886 - val_accuracy: 0.5974
Epoch 116/1000
178/178 [==============================] - 0s 121us/step - loss: 0.5192 - accuracy: 0.7753 - val_loss: 0.9056 - val_accuracy: 0.5974
Epoch 117/1000
178/178 [==============================] - 0s 105us/step - loss: 0.5495 - accuracy: 0.7584 - val_loss: 0.6954 - val_accuracy: 0.5974
Epoch 118/1000
178/178 [==============================] - 0s 95us/step - loss: 0.5247 - accuracy: 0.7640 - val_loss: 0.8494 - val_accuracy: 0.5974
Epoch 119/1000
178/178 [==============================] - 0s 75us/step - loss: 0.5400 - accuracy: 0.7584 - val_lo

178/178 [==============================] - 0s 84us/step - loss: 0.5096 - accuracy: 0.7697 - val_loss: 0.6736 - val_accuracy: 0.6104
Epoch 169/1000
178/178 [==============================] - 0s 75us/step - loss: 0.4926 - accuracy: 0.7921 - val_loss: 0.6820 - val_accuracy: 0.5974
Epoch 170/1000
178/178 [==============================] - 0s 86us/step - loss: 0.4970 - accuracy: 0.7584 - val_loss: 0.6778 - val_accuracy: 0.5974
Epoch 171/1000
178/178 [==============================] - 0s 76us/step - loss: 0.4833 - accuracy: 0.7921 - val_loss: 0.6885 - val_accuracy: 0.5974
Epoch 172/1000
178/178 [==============================] - 0s 84us/step - loss: 0.4770 - accuracy: 0.7865 - val_loss: 0.6862 - val_accuracy: 0.6104
Epoch 173/1000
178/178 [==============================] - 0s 83us/step - loss: 0.4747 - accuracy: 0.7865 - val_loss: 0.6886 - val_accuracy: 0.5974
Epoch 174/1000
178/178 [==============================] - 0s 75us/step - loss: 0.4996 - accuracy: 0.7753 - val_loss: 0.6827 - val_acc

Epoch 224/1000
178/178 [==============================] - 0s 94us/step - loss: 0.4786 - accuracy: 0.7809 - val_loss: 0.6804 - val_accuracy: 0.6104
Epoch 225/1000
178/178 [==============================] - 0s 86us/step - loss: 0.4630 - accuracy: 0.7697 - val_loss: 0.7050 - val_accuracy: 0.5974
Epoch 226/1000
178/178 [==============================] - 0s 85us/step - loss: 0.4821 - accuracy: 0.7809 - val_loss: 0.7054 - val_accuracy: 0.5974
Epoch 227/1000
178/178 [==============================] - 0s 93us/step - loss: 0.4693 - accuracy: 0.7584 - val_loss: 0.7081 - val_accuracy: 0.5974
Epoch 228/1000
178/178 [==============================] - 0s 124us/step - loss: 0.5081 - accuracy: 0.7809 - val_loss: 0.7169 - val_accuracy: 0.5974
Epoch 229/1000
178/178 [==============================] - 0s 143us/step - loss: 0.4874 - accuracy: 0.7809 - val_loss: 0.8159 - val_accuracy: 0.5974
Epoch 230/1000
178/178 [==============================] - 0s 97us/step - loss: 0.5029 - accuracy: 0.7753 - val_loss:

...

178/178 [==============================] - 0s 86us/step - loss: 0.3751 - accuracy: 0.8371 - val_loss: 0.7308 - val_accuracy: 0.5974
Epoch 777/1000
178/178 [==============================] - 0s 82us/step - loss: 0.4054 - accuracy: 0.8202 - val_loss: 0.7210 - val_accuracy: 0.5974
Epoch 778/1000
178/178 [==============================] - 0s 80us/step - loss: 0.3790 - accuracy: 0.8371 - val_loss: 0.7242 - val_accuracy: 0.5974
Epoch 779/1000
178/178 [==============================] - 0s 148us/step - loss: 0.3990 - accuracy: 0.8202 - val_loss: 0.8273 - val_accuracy: 0.5974
Epoch 780/1000
178/178 [==============================] - 0s 106us/step - loss: 0.4131 - accuracy: 0.8090 - val_loss: 0.7286 - val_accuracy: 0.5974
Epoch 781/1000
178/178 [==============================] - 0s 121us/step - loss: 0.3833 - accuracy: 0.8427 - val_loss: 1.2355 - val_accuracy: 0.5844
Epoch 782/1000
178/178 [==============================] - 0s 98us/step - loss: 0.4999 - accuracy: 0.8146 - val_loss: 1.0066 - val_

178/178 [==============================] - 0s 175us/step - loss: 0.3987 - accuracy: 0.8315 - val_loss: 0.8866 - val_accuracy: 0.5974
Epoch 832/1000
178/178 [==============================] - 0s 199us/step - loss: 0.4398 - accuracy: 0.8090 - val_loss: 0.7532 - val_accuracy: 0.5844
Epoch 833/1000
178/178 [==============================] - 0s 190us/step - loss: 0.3782 - accuracy: 0.8315 - val_loss: 0.8036 - val_accuracy: 0.5844
Epoch 834/1000
178/178 [==============================] - 0s 194us/step - loss: 0.3745 - accuracy: 0.8315 - val_loss: 0.7269 - val_accuracy: 0.5974
Epoch 835/1000
178/178 [==============================] - 0s 168us/step - loss: 0.3739 - accuracy: 0.8371 - val_loss: 0.7304 - val_accuracy: 0.5974
Epoch 836/1000
178/178 [==============================] - 0s 157us/step - loss: 0.4151 - accuracy: 0.8315 - val_loss: 0.7515 - val_accuracy: 0.5974
Epoch 837/1000
178/178 [==============================] - 0s 127us/step - loss: 0.3879 - accuracy: 0.8315 - val_loss: 0.7204 - 

178/178 [==============================] - 0s 96us/step - loss: 0.4037 - accuracy: 0.8090 - val_loss: 0.7776 - val_accuracy: 0.5844
Epoch 887/1000
178/178 [==============================] - 0s 141us/step - loss: 0.3959 - accuracy: 0.8427 - val_loss: 0.7334 - val_accuracy: 0.5844
Epoch 888/1000
178/178 [==============================] - 0s 130us/step - loss: 0.3677 - accuracy: 0.8258 - val_loss: 0.9237 - val_accuracy: 0.5844
Epoch 889/1000
178/178 [==============================] - 0s 115us/step - loss: 0.4260 - accuracy: 0.8258 - val_loss: 0.7357 - val_accuracy: 0.5844
Epoch 890/1000
178/178 [==============================] - 0s 137us/step - loss: 0.3496 - accuracy: 0.8427 - val_loss: 0.9071 - val_accuracy: 0.5844
Epoch 891/1000
178/178 [==============================] - 0s 254us/step - loss: 0.3630 - accuracy: 0.8427 - val_loss: 0.7409 - val_accuracy: 0.5844
Epoch 892/1000
178/178 [==============================] - 0s 169us/step - loss: 0.3767 - accuracy: 0.8258 - val_loss: 0.7319 - v

178/178 [==============================] - 0s 99us/step - loss: 0.4306 - accuracy: 0.8315 - val_loss: 0.7684 - val_accuracy: 0.5714
Epoch 942/1000
178/178 [==============================] - 0s 124us/step - loss: 0.3997 - accuracy: 0.8315 - val_loss: 0.7604 - val_accuracy: 0.5844
Epoch 943/1000
178/178 [==============================] - 0s 88us/step - loss: 0.3522 - accuracy: 0.8371 - val_loss: 0.7462 - val_accuracy: 0.5844
Epoch 944/1000
178/178 [==============================] - 0s 87us/step - loss: 0.3783 - accuracy: 0.8202 - val_loss: 0.7419 - val_accuracy: 0.5974
Epoch 945/1000
178/178 [==============================] - 0s 94us/step - loss: 0.3428 - accuracy: 0.8539 - val_loss: 0.7708 - val_accuracy: 0.5844
Epoch 946/1000
178/178 [==============================] - 0s 105us/step - loss: 0.3472 - accuracy: 0.8427 - val_loss: 0.9745 - val_accuracy: 0.5714
Epoch 947/1000
178/178 [==============================] - 0s 128us/step - loss: 0.3775 - accuracy: 0.8258 - val_loss: 0.7372 - val_

178/178 [==============================] - 0s 81us/step - loss: 0.3488 - accuracy: 0.8371 - val_loss: 0.8115 - val_accuracy: 0.5714
Epoch 997/1000
178/178 [==============================] - 0s 98us/step - loss: 0.4191 - accuracy: 0.8315 - val_loss: 0.7432 - val_accuracy: 0.5844
Epoch 998/1000
178/178 [==============================] - 0s 92us/step - loss: 0.3532 - accuracy: 0.8371 - val_loss: 0.7687 - val_accuracy: 0.5844
Epoch 999/1000
178/178 [==============================] - 0s 84us/step - loss: 0.3680 - accuracy: 0.8315 - val_loss: 0.7372 - val_accuracy: 0.5844
Epoch 1000/1000
178/178 [==============================] - 0s 94us/step - loss: 0.3655 - accuracy: 0.8315 - val_loss: 0.7383 - val_accuracy: 0.5844


In [41]:
acc_test_sel1 = model_sel1.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel1*100))

77/77 [==============================] - 0s 69us/step
test accuracy: 58.44%


In [42]:
#### improve neural network
model_sel2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [43]:
model_sel2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [44]:
model_sel2.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 744us/step - loss: 12.8127 - accuracy: 0.5056 - val_loss: 2.0977 - val_accuracy: 0.5455
Epoch 2/1000
178/178 [==============================] - 0s 51us/step - loss: 1.8016 - accuracy: 0.5674 - val_loss: 0.7550 - val_accuracy: 0.5455
Epoch 3/1000
178/178 [==============================] - 0s 55us/step - loss: 0.9078 - accuracy: 0.5337 - val_loss: 0.6848 - val_accuracy: 0.5455
Epoch 4/1000
178/178 [==============================] - 0s 59us/step - loss: 0.6825 - accuracy: 0.5393 - val_loss: 0.6839 - val_accuracy: 0.5455
Epoch 5/1000
178/178 [==============================] - 0s 56us/step - loss: 0.6789 - accuracy: 0.4831 - val_loss: 0.6835 - val_accuracy: 0.5455
Epoch 6/1000
178/178 [==============================] - 0s 61us/step - loss: 0.6767 - accuracy: 0.4775 - val_loss: 0.6826 - val_accuracy: 0.5584
Epoch 7/1000
178/178 [==============================] - 0s 62us/step - loss: 0.6742

Epoch 57/1000
178/178 [==============================] - 0s 67us/step - loss: 0.6111 - accuracy: 0.7528 - val_loss: 0.6744 - val_accuracy: 0.5325
Epoch 58/1000
178/178 [==============================] - 0s 82us/step - loss: 0.6035 - accuracy: 0.7528 - val_loss: 0.6742 - val_accuracy: 0.5325
Epoch 59/1000
178/178 [==============================] - 0s 71us/step - loss: 0.6049 - accuracy: 0.7416 - val_loss: 0.6741 - val_accuracy: 0.5325
Epoch 60/1000
178/178 [==============================] - 0s 61us/step - loss: 0.6056 - accuracy: 0.7640 - val_loss: 0.6758 - val_accuracy: 0.5455
Epoch 61/1000
178/178 [==============================] - 0s 56us/step - loss: 0.6012 - accuracy: 0.7584 - val_loss: 0.6760 - val_accuracy: 0.5455
Epoch 62/1000
178/178 [==============================] - 0s 68us/step - loss: 0.6048 - accuracy: 0.7528 - val_loss: 0.6742 - val_accuracy: 0.5325
Epoch 63/1000
178/178 [==============================] - 0s 67us/step - loss: 0.6009 - accuracy: 0.7640 - val_loss: 0.6755 -

Epoch 113/1000
178/178 [==============================] - 0s 92us/step - loss: 0.5702 - accuracy: 0.7640 - val_loss: 0.6812 - val_accuracy: 0.5714
Epoch 114/1000
178/178 [==============================] - 0s 127us/step - loss: 0.5547 - accuracy: 0.7809 - val_loss: 0.6802 - val_accuracy: 0.5714
Epoch 115/1000
178/178 [==============================] - 0s 246us/step - loss: 0.5537 - accuracy: 0.7697 - val_loss: 0.6814 - val_accuracy: 0.5714
Epoch 116/1000
178/178 [==============================] - 0s 214us/step - loss: 0.5536 - accuracy: 0.7921 - val_loss: 0.6821 - val_accuracy: 0.5714
Epoch 117/1000
178/178 [==============================] - 0s 224us/step - loss: 0.5524 - accuracy: 0.7640 - val_loss: 0.6888 - val_accuracy: 0.5455
Epoch 118/1000
178/178 [==============================] - 0s 178us/step - loss: 0.5522 - accuracy: 0.7640 - val_loss: 0.6810 - val_accuracy: 0.5714
Epoch 119/1000
178/178 [==============================] - 0s 184us/step - loss: 0.5611 - accuracy: 0.7640 - val_l

178/178 [==============================] - 0s 123us/step - loss: 0.5391 - accuracy: 0.7416 - val_loss: 0.6871 - val_accuracy: 0.5584
Epoch 169/1000
178/178 [==============================] - 0s 93us/step - loss: 0.5239 - accuracy: 0.7697 - val_loss: 0.6828 - val_accuracy: 0.5714
Epoch 170/1000
178/178 [==============================] - 0s 100us/step - loss: 0.5185 - accuracy: 0.7921 - val_loss: 0.6819 - val_accuracy: 0.5584
Epoch 171/1000
178/178 [==============================] - 0s 61us/step - loss: 0.5223 - accuracy: 0.7809 - val_loss: 0.6881 - val_accuracy: 0.5714
Epoch 172/1000
178/178 [==============================] - 0s 64us/step - loss: 0.5137 - accuracy: 0.7921 - val_loss: 0.6996 - val_accuracy: 0.5584
Epoch 173/1000
178/178 [==============================] - 0s 70us/step - loss: 0.5627 - accuracy: 0.7360 - val_loss: 0.7185 - val_accuracy: 0.5455
Epoch 174/1000
178/178 [==============================] - 0s 61us/step - loss: 0.5381 - accuracy: 0.7640 - val_loss: 0.6810 - val_a

Epoch 224/1000
178/178 [==============================] - 0s 131us/step - loss: 0.4941 - accuracy: 0.7865 - val_loss: 0.6884 - val_accuracy: 0.5714
Epoch 225/1000
178/178 [==============================] - 0s 160us/step - loss: 0.4948 - accuracy: 0.7809 - val_loss: 0.6994 - val_accuracy: 0.5455
Epoch 226/1000
178/178 [==============================] - 0s 129us/step - loss: 0.5119 - accuracy: 0.7584 - val_loss: 0.7360 - val_accuracy: 0.5584
Epoch 227/1000
178/178 [==============================] - 0s 94us/step - loss: 0.5058 - accuracy: 0.7809 - val_loss: 0.6980 - val_accuracy: 0.5584
Epoch 228/1000
178/178 [==============================] - 0s 115us/step - loss: 0.4901 - accuracy: 0.7809 - val_loss: 0.6948 - val_accuracy: 0.5714
Epoch 229/1000
178/178 [==============================] - 0s 94us/step - loss: 0.5047 - accuracy: 0.7584 - val_loss: 0.6938 - val_accuracy: 0.5714
Epoch 230/1000
178/178 [==============================] - 0s 103us/step - loss: 0.5008 - accuracy: 0.7697 - val_lo

...

178/178 [==============================] - 0s 94us/step - loss: 0.4080 - accuracy: 0.8090 - val_loss: 0.7739 - val_accuracy: 0.5974
Epoch 779/1000
178/178 [==============================] - 0s 110us/step - loss: 0.4091 - accuracy: 0.8202 - val_loss: 0.7356 - val_accuracy: 0.6104
Epoch 780/1000
178/178 [==============================] - 0s 99us/step - loss: 0.3930 - accuracy: 0.8258 - val_loss: 0.7693 - val_accuracy: 0.5974
Epoch 781/1000
178/178 [==============================] - 0s 92us/step - loss: 0.4246 - accuracy: 0.8034 - val_loss: 0.7386 - val_accuracy: 0.6104
Epoch 782/1000
178/178 [==============================] - 0s 85us/step - loss: 0.3983 - accuracy: 0.8202 - val_loss: 1.0220 - val_accuracy: 0.5974
Epoch 783/1000
178/178 [==============================] - 0s 70us/step - loss: 0.6042 - accuracy: 0.7809 - val_loss: 0.7106 - val_accuracy: 0.5974
Epoch 784/1000
178/178 [==============================] - 0s 74us/step - loss: 0.4167 - accuracy: 0.8202 - val_loss: 0.7317 - val_ac

178/178 [==============================] - 0s 67us/step - loss: 0.4300 - accuracy: 0.8202 - val_loss: 0.7294 - val_accuracy: 0.6104
Epoch 834/1000
178/178 [==============================] - 0s 57us/step - loss: 0.3908 - accuracy: 0.8202 - val_loss: 0.8591 - val_accuracy: 0.5974
Epoch 835/1000
178/178 [==============================] - 0s 56us/step - loss: 0.4172 - accuracy: 0.7978 - val_loss: 0.7585 - val_accuracy: 0.5844
Epoch 836/1000
178/178 [==============================] - 0s 62us/step - loss: 0.3849 - accuracy: 0.8371 - val_loss: 0.8371 - val_accuracy: 0.6104
Epoch 837/1000
178/178 [==============================] - 0s 93us/step - loss: 0.4420 - accuracy: 0.7865 - val_loss: 0.7359 - val_accuracy: 0.6104
Epoch 838/1000
178/178 [==============================] - 0s 67us/step - loss: 0.3863 - accuracy: 0.8258 - val_loss: 0.7444 - val_accuracy: 0.6104
Epoch 839/1000
178/178 [==============================] - 0s 84us/step - loss: 0.3839 - accuracy: 0.8483 - val_loss: 0.7550 - val_acc

Epoch 889/1000
178/178 [==============================] - 0s 74us/step - loss: 0.3906 - accuracy: 0.8315 - val_loss: 0.7438 - val_accuracy: 0.6104
Epoch 890/1000
178/178 [==============================] - 0s 70us/step - loss: 0.3779 - accuracy: 0.8483 - val_loss: 0.7566 - val_accuracy: 0.6104
Epoch 891/1000
178/178 [==============================] - 0s 62us/step - loss: 0.3813 - accuracy: 0.8371 - val_loss: 0.8201 - val_accuracy: 0.5974
Epoch 892/1000
178/178 [==============================] - 0s 74us/step - loss: 0.4057 - accuracy: 0.8034 - val_loss: 0.7770 - val_accuracy: 0.5844
Epoch 893/1000
178/178 [==============================] - 0s 61us/step - loss: 0.3809 - accuracy: 0.8483 - val_loss: 0.7574 - val_accuracy: 0.6104
Epoch 894/1000
178/178 [==============================] - 0s 76us/step - loss: 0.3782 - accuracy: 0.8315 - val_loss: 0.9163 - val_accuracy: 0.5974
Epoch 895/1000
178/178 [==============================] - 0s 85us/step - loss: 0.3894 - accuracy: 0.8258 - val_loss: 0

178/178 [==============================] - 0s 82us/step - loss: 0.4346 - accuracy: 0.8034 - val_loss: 0.7613 - val_accuracy: 0.6104
Epoch 945/1000
178/178 [==============================] - 0s 85us/step - loss: 0.3622 - accuracy: 0.8427 - val_loss: 0.8170 - val_accuracy: 0.5974
Epoch 946/1000
178/178 [==============================] - 0s 67us/step - loss: 0.4384 - accuracy: 0.7865 - val_loss: 0.7668 - val_accuracy: 0.6104
Epoch 947/1000
178/178 [==============================] - 0s 68us/step - loss: 0.3804 - accuracy: 0.8202 - val_loss: 0.7630 - val_accuracy: 0.6104
Epoch 948/1000
178/178 [==============================] - 0s 73us/step - loss: 0.3798 - accuracy: 0.8258 - val_loss: 0.7658 - val_accuracy: 0.6104
Epoch 949/1000
178/178 [==============================] - 0s 63us/step - loss: 0.4000 - accuracy: 0.8034 - val_loss: 0.7777 - val_accuracy: 0.6104
Epoch 950/1000
178/178 [==============================] - 0s 65us/step - loss: 0.3756 - accuracy: 0.8202 - val_loss: 0.8030 - val_acc

Epoch 1000/1000
178/178 [==============================] - 0s 82us/step - loss: 0.3549 - accuracy: 0.8427 - val_loss: 0.8644 - val_accuracy: 0.5974


In [45]:
acc_test_sel2 = model_sel2.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel2*100))

77/77 [==============================] - 0s 102us/step
test accuracy: 59.74%


In [46]:
############ Logistic Regression ############

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [48]:
###### logistics
log = LogisticRegression(random_state=123)
log.fit(X_train, y_train)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [49]:
y_pred = log.predict(X_test)
print('combination logistic test accuracy %.2f%%' % (log.score(X_test, y_test)*100))

combination logistic test accuracy 61.04%


In [50]:
############## Random Forest ##############

In [51]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=123)
rf.fit(X_train,y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [52]:
from sklearn.metrics import accuracy_score
y_pred = rf.predict(X_test)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test, y_pred)*100))

combination test accuracy: 63.64%


In [53]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [54]:
## random forest model with CV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv = RandomForestClassifier(n_estimators=100, random_state=123)

accs = cross_val_score(estimator=rf, X=X_train, y=y_train, cv=5)
print(accs)
print(accs.mean())

[0.83333333 0.80555556 0.69444444 0.80555556 0.73529412]
0.7748366013071895
